In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "lab04") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [2]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer,StringIndexer,OneHotEncoder
import pyspark.sql.functions as f_
from pyspark.sql.types import FloatType, StructType, StructField, IntegerType, StringType, ArrayType,TimestampType
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
import json
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml.classification import RandomForestClassificationModel


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
from urllib.parse import unquote

In [6]:
@f_.udf(StringType())
def url_cyr(url):
    return str(unquote(url))

### Kafka

In [34]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time is :", current_time)

Current Time is : 19:17:07


In [35]:
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_ilya.chulyukin",
    "checkpointLocation":"/tmp/chk_/{n}".format(n='ilya.chulyukin'),
    "startingOffsets": "earliest"
}
kafka_sdf = spark.read.format("kafka").options(**read_kafka_params).load()

In [36]:
kafka_sdf.count()

5000

In [37]:
json_schema = "map<string, array<struct<url:string,timestamp:string>>>"
kafka_in_tabular_data=kafka_sdf.select(f_.get_json_object(f_.col("value").cast("string"), "$.uid").alias("uid"),
                f_.concat(f_.lit('''{"visits": '''),f_.get_json_object(f_.col("value").cast("string"), "$.visits"),f_.lit('''}''')).alias("user_json")
                ).withColumn('json', f_.from_json(f_.col('user_json'), json_schema))\
    .select('uid', f_.explode('json'))\
    .select('uid',f_.explode('value'))\
    .select('uid',f_.col('col').url.alias('url'), f_.col('col').timestamp.alias('timestamp'))\
    .select( 'uid',f_.col('timestamp').alias('ts_original'),f_.to_timestamp(f_.from_unixtime(f_.col('timestamp')/1000)).alias('timestamp'), url_cyr(f_.col('url')).alias('url'))\
    .select('*'
                ,f_.regexp_replace(f_.lower(f_.split(f_.col('url'),'/').getItem(2)),'www.','').alias('domain')
                ,f_.when((f_.length(f_.split(f_.col('url'),'\\?').getItem(0))  >  f_.coalesce(f_.length(f_.split(f_.col('url'),'\\?').getItem(1)),f_.lit(0))), f_.split(f_.col('url'),'\\?').getItem(0) ).otherwise(f_.col('url')).alias('url2'))\
    .select('*',f_.trim(f_.lower(f_.regexp_replace(f_.regexp_replace('url2',r'[^\pL\p{Space}]',' ' ),'[ ]+',' '))).alias('url_words'))\
            .withColumn("lag_ts",f_.lag('timestamp').over(Window.partitionBy("uid").orderBy(f_.col('timestamp'))))\
            .withColumn("lag_domain",f_.lag('domain').over(Window.partitionBy("uid").orderBy(f_.col('timestamp'))))\
    .select('*'
                ,(f_.col('timestamp').cast('long') - f_.col('lag_ts').cast('long')).alias('timediff')
            ,f_.when(((f_.col('lag_domain').isNotNull()) & (f_.col('lag_domain') != f_.col('domain'))),f_.lit(1)).otherwise(f_.lit(0)).alias('change_site_flg')
           )
   

In [38]:
kafka_in_grouped_tabular_data=kafka_in_tabular_data.groupBy('uid').agg(
     f_.concat_ws( ' ',f_.collect_list(f_.col('url_words'))).alias('urls_w')
    ,f_.concat_ws( ' ',f_.collect_list(f_.col('domain'))).alias('domains')
    ,f_.max('timestamp').cast('long').alias('max_ts')
    ,f_.min('timestamp').cast('long').alias('min_ts')
    ,f_.avg('timestamp').cast('long').alias('avg_ts')
    ,f_.max('timediff').alias('max_tdiff')
    ,f_.min('timediff').alias('min_tdiff')
    ,f_.avg('timediff').alias('avg_tdiff')
    ,f_.countDistinct('domain').alias('domain_cnt')
    ,f_.sum('change_site_flg').alias('change_site_cnt')
    ,f_.count('*').alias('visit_cnt')
                                                )

In [39]:
kafka_in_grouped_tabular_data.count()

5000

In [40]:
%%time
kafka_tabular_tfidf_data=calc.transform(kafka_in_grouped_tabular_data)
test_data=assembler.transform(kafka_tabular_tfidf_data).select('uid','features')
model_load=RandomForestClassificationModel.load('lr_01_dup.sav')
test_predictions=model_load.transform(test_data)

CPU times: user 36.8 ms, sys: 10 ms, total: 46.9 ms
Wall time: 27 s


In [41]:
label_decode=spark.createDataFrame([[float(i),v.split('_')[0],v.split('_')[1]] for i,v in enumerate(si_label.labels)],schema='prediction:float, gender:string,age:string')
result=test_predictions.join(label_decode,['prediction'],'left').select('uid','gender','age')


In [42]:
def write_kafka(topic, data):
    kafka_params = {"kafka.bootstrap.servers": "spark-master-1.newprolab.com:6667"}
    kafka_doc = f_.to_json(f_.struct(f_.col("*")))
    raw = data \
        .select(kafka_doc.alias("value")) \
        .withColumn("topic", f_.lit(topic))
    return raw

In [43]:
%%time
write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ilya.chulyukin",
   "checkpointLocation":"/tmp/chk_/{n}".format(n='ilya.chulyukin')
}
write_kafka('ilya.chulyukin',result).write.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "/tmp/chk_/{n}".format(n='ilya.chulyukin')).save()

CPU times: user 21.8 ms, sys: 14.8 ms, total: 36.6 ms
Wall time: 1min 12s


In [44]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time is :", current_time)

Current Time is : 19:19:13


In [ ]:
# ! hdfs dfs -ls

In [53]:
# ! hdfs dfs -get -f hdfs://spark-master-1.newprolab.com:8020/user/grigoriy.propirnyy/lr_22_dup.sav